In [6]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [7]:
client = bigquery.Client()

In [8]:
query = """ 
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_abr_siape_pensionista_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2021_abr_siape_pensionista_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2022_abr_siape_pensionista_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_abr_siape_pensionista_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_abr_siape_pensionista_cadastro` 
        """  


In [9]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_pensao,total
0,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),1
1,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),1
2,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),1
3,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),1
4,2024,4,Filho (a) / Menor sob guarda / Enteado (a),1
...,...,...,...,...
1447457,2020,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),1
1447458,2020,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),1
1447459,2020,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),1
1447460,2020,4,Filho (a) / Menor sob guarda / Enteado (a),1


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447462 entries, 0 to 1447461
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   ano          1447462 non-null  Int64 
 1   mes          1447462 non-null  Int64 
 2   tipo_pensao  1447462 non-null  object
 3   total        1447462 non-null  Int64 
dtypes: Int64(3), object(1)
memory usage: 48.3+ MB


In [23]:
df['tipo_pensao'].unique()

array(['(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a)',
       'Filho (a) / Menor sob guarda / Enteado (a)', 'Pai / Mae',
       'Outros', 'Pessoa designada', 'Irmao (a)', 'Neto (a)', 'Inválido'],
      dtype=object)

In [22]:
df[df['tipo_pensao']=='Inválido'] # Corresponde a 0.82% do total

,ano,mes,tipo_pensao,total
22766,2024,4,Inválido,1
23037,2024,4,Inválido,1
23143,2024,4,Inválido,1
23267,2024,4,Inválido,1
23699,2024,4,Inválido,1
...,...,...,...,...
1407263,2020,4,Inválido,1
1407268,2020,4,Inválido,1
1407301,2020,4,Inválido,1
1407349,2020,4,Inválido,1


In [24]:

df = df.drop(df.index[df['tipo_pensao']=='Inválido']) # Esses dados que foram descartados correspondem a 0.82% do total 

In [26]:
df['tipo_pensao'].unique()

array(['(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a)',
       'Filho (a) / Menor sob guarda / Enteado (a)', 'Pai / Mae',
       'Outros', 'Pessoa designada', 'Irmao (a)', 'Neto (a)'],
      dtype=object)

In [28]:
pensoes = {'(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a)':'Cônjuge ou ex-cônjuge',
'Filho (a) / Menor sob guarda / Enteado (a)':"Filhos, netos, enteados ou menores em tutela",
'Pai / Mae':'Pais',
'Outros':'Outros',
'Pessoa designada':'Pessoa designada',
'Irmao (a)':'Irmão (ã)',
'Neto (a)':'Filhos, netos, enteados ou menores em tutela"'}

In [29]:
def categorizando(x):
    if x in pensoes:
        return pensoes[x]


In [30]:
df['tipo_pensao_agrupado'] = df['tipo_pensao'].apply(categorizando)

In [31]:
df= df[['ano', 'mes', 'tipo_pensao','tipo_pensao_agrupado', 'total']]
df

,ano,mes,tipo_pensao,tipo_pensao_agrupado,total
0,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
1,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
2,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
3,2024,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
4,2024,4,Filho (a) / Menor sob guarda / Enteado (a),"Filhos, netos, enteados ou menores em tutela",1
...,...,...,...,...,...
1447457,2020,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
1447458,2020,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
1447459,2020,4,(Ex) conjuge / (Ex) companheiro (a) / Viuvo (a),Cônjuge ou ex-cônjuge,1
1447460,2020,4,Filho (a) / Menor sob guarda / Enteado (a),"Filhos, netos, enteados ou menores em tutela",1


In [37]:
df['tipo_pensao_agrupado'].unique()

array(['Cônjuge ou ex-cônjuge',
       'Filhos, netos, enteados ou menores em tutela', 'Pais', 'Outros',
       'Pessoa designada', 'Irmão (ã)',
       'Filhos, netos, enteados ou menores em tutela"'], dtype=object)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1435553 entries, 0 to 1447461
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   ano                   1435553 non-null  Int64 
 1   mes                   1435553 non-null  Int64 
 2   tipo_pensao           1435553 non-null  object
 3   tipo_pensao_agrupado  1435553 non-null  object
 4   total                 1435553 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 69.8+ MB


In [34]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [35]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),        
        bigquery.SchemaField('tipo_pensao','STRING',description='Tipo de pensão'),
        bigquery.SchemaField('tipo_pensao_agrupado','STRING',description='Categorização das pensões'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')
        ]

In [36]:
table_ref = dataset_ref.table('SIAPE_pensionistas_total') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=4ad99ba7-fbc6-4801-86d1-ed134f699d9d>